# 0.저장

In [ ]:
# from utils import load_json_from_url, load_config
# import json

# cfg = load_config("config.yaml")
# data_list = load_json_from_url(cfg['path']['zsre_train_path'])

# with open('zsre_train.jsonl', 'w') as jsonl_file:
#     for item in data_list:
#         json.dump(item, jsonl_file)
#         jsonl_file.write('\n')

In [ ]:
# cfg = load_config("config.yaml")
# data_list = load_json_from_url(cfg['path']['zsre_eval_path'])

# with open('zsre_eval.jsonl', 'w') as jsonl_file:
#     for item in data_list:
#         json.dump(item, jsonl_file)
#         jsonl_file.write('\n')

In [5]:
# cfg = load_config("config.yaml")
# data_list = load_json_from_url(cfg['path']['counterfact_path'])

# with open('counterfact.jsonl', 'w') as jsonl_file:
#     for item in data_list:
#         json.dump(item, jsonl_file)
#         jsonl_file.write('\n')

JSON 데이터 도착


In [6]:
# from utils import load_jsonl_from_path, load_config
# cfg = load_config("config.yaml")
# data_list = load_jsonl_from_path(cfg['path']['fever_path'])

# with open('fever.jsonl', 'w') as jsonl_file:
#     for item in data_list:
#         json.dump(item, jsonl_file)
#         jsonl_file.write('\n')

# 1.zsRE

zsRE는 어디서 시작했는가?
1. wikidata를 기반으로 "Zero-Shot Relation Extraction via Reading Comprehension; Omer Levy et al." 에서 처음 제작(2017)
    * 약 3000만개
    * tab으로 나뉜 txt 데이터(utf-8, 500MB)
    * 본 목적은 MRC를 위해 제작됨
    * 120개의 relation
    * zsRE_origin.ipynb 참조

2. MEND 저자가 해당 데이터를 재구성 (BART 모델을 통한 응답 + locality 측정을 위한 데이터 추가)
    * 해당 데이터를 latin1 타입으로 인코딩해서 올려놓고 사용 코드는 전처리된 jsonl 파일로 사용 -> 뜯다가 gg (jsonl은 utf-8기반)

3. ROME에서 (2) 데이터를 잘 정리해서 올려놓음 굿~

In [2]:
from utils import load_json_from_url, load_config

cfg = load_config("config.yaml")
zsre_data = load_json_from_url(cfg['path']['zsre_train_path'])
zsre_data_eval = load_json_from_url(cfg['path']['zsre_eval_path'])

JSON 데이터 도착
JSON 데이터 도착


In [3]:
# with open('zsre_train.jsonl', 'w') as jsonl_file:
#     for item in zsre_data:
#         json.dump(item, jsonl_file)
#         jsonl_file.write('\n')

In [4]:
# with open('zsre_eval.jsonl', 'w') as jsonl_file:
#     for item in zsre_data_eval:
#         json.dump(item, jsonl_file)
#         jsonl_file.write('\n')

In [36]:
num_train = len(zsre_data)
num_eval = len(zsre_data_eval)
print(f'zsre size: {num_train}개')
print(f'zsre eval size: {num_eval}개')
print(f'total size: {num_train + num_eval}개')


zsre size: 163196개
zsre eval size: 19086개
total size: 182282개


In [9]:
zsre_data[1]

# relation이 정형화되는지 확인

{'subject': 'Colt King Cobra',
 'src': 'Which company manufactured Colt King Cobra?',
 'pred': "Colt's Manufacturing",
 'rephrase': 'Who was the manufacturer of the Colt King Cobra?',
 'alt': ' Colt Motorcycles',
 'answers': ["Colt's Manufacturing Company"],
 'loc': 'nq question: where are the winter olympics going to be',
 'loc_ans': 'Seoul',
 'cond': "Colt's Manufacturing >>  Colt Motorcycles || Which company manufactured Colt King Cobra?"}

In [14]:
from collections import Counter
num = []
for row in zsre_data:
    num.append(len(row['answers']))
    
counter = Counter(num)
counter

# 복수정답인지 다 맞춰야되는지 확인

Counter({1: 159468,
         2: 2733,
         6: 47,
         3: 605,
         4: 232,
         5: 82,
         7: 11,
         8: 12,
         9: 3,
         12: 1,
         10: 1,
         11: 1})

# 2.CounterFact

CounterFact 데이터는 어디에서 왔는가?
1. TREx 데이터 만들어짐
2. LAMA에서 T-REx를 41개의 relation으로 뽑음 "Language Models as Knowledge Bases?; Fabio Petroni et al."
3. T-REx 데이터 뽑아서 ParaRel 데이터 만듦 (그중 좋지 않은 relation 3개 빼서 38개됨)
4. ParaRel 데이터 뽑아서 CounterFact 만듦 (34개 relation)

CounterFact는 **ParaRel** dataset에서 파생된 데이터 형태

In [15]:
from utils import load_json_from_url, load_config

cfg = load_config("config.yaml")
counterfact_data = load_json_from_url(cfg['path']['counterfact_path'])

JSON 데이터 도착


In [16]:
print(f'counterfact size: {len(counterfact_data)}개')

counterfact size: 21919개


In [17]:
counterfact_data[0]

{'case_id': 0,
 'pararel_idx': 2796,
 'requested_rewrite': {'prompt': 'The mother tongue of {} is',
  'relation_id': 'P103',
  'target_new': {'str': 'English', 'id': 'Q1860'},
  'target_true': {'str': 'French', 'id': 'Q150'},
  'subject': 'Danielle Darrieux'},
 'paraphrase_prompts': ['Shayna does this and Yossel goes still and dies. Danielle Darrieux, a native',
  'An album was recorded for Capitol Nashville but never released. Danielle Darrieux spoke the language'],
 'neighborhood_prompts': ['The mother tongue of Léon Blum is',
  'The native language of Montesquieu is',
  'François Bayrou, a native',
  'The native language of Raymond Barre is',
  'Michel Rocard is a native speaker of',
  'Jacques Chaban-Delmas is a native speaker of',
  'The native language of François Bayrou is',
  'Maurice Genevoix, speaker of',
  'The mother tongue of François Bayrou is',
  'Melchior de Vogüé, speaker of'],
 'attribute_prompts': ['J.\xa0R.\xa0R. Tolkien is a native speaker of',
  'The mother tongue

In [23]:
counterfact_data[0].keys()

dict_keys(['case_id', 'pararel_idx', 'requested_rewrite', 'paraphrase_prompts', 'neighborhood_prompts', 'attribute_prompts', 'generation_prompts'])

In [26]:
counterfact_data[0]['requested_rewrite']

{'prompt': 'The mother tongue of {} is',
 'relation_id': 'P103',
 'target_new': {'str': 'English', 'id': 'Q1860'},
 'target_true': {'str': 'French', 'id': 'Q150'},
 'subject': 'Danielle Darrieux'}

In [30]:
from collections import Counter
relation = []
for row in counterfact_data:
    relation.append(row['requested_rewrite']['relation_id'])
    
counter = Counter(relation)
counter

Counter({'P103': 919,
         'P140': 430,
         'P1303': 513,
         'P17': 875,
         'P190': 621,
         'P740': 774,
         'P178': 579,
         'P495': 904,
         'P127': 433,
         'P641': 318,
         'P176': 911,
         'P413': 952,
         'P364': 751,
         'P39': 476,
         'P276': 613,
         'P463': 163,
         'P159': 756,
         'P20': 816,
         'P136': 847,
         'P106': 821,
         'P30': 959,
         'P937': 846,
         'P449': 794,
         'P27': 958,
         'P1412': 924,
         'P101': 545,
         'P19': 779,
         'P37': 891,
         'P138': 279,
         'P131': 714,
         'P407': 216,
         'P108': 350,
         'P36': 139,
         'P264': 53})

In [33]:
print(f'relation_id 개수: {len(set(counter.keys()))}개')

relation_id 개수: 34개


# 3.ParaRel

pararel-main/data/pattern_data/graphs_json 확인

# 4.FEVER


page : https://fever.ai/dataset/fever.html

In [1]:
from utils import load_jsonl_from_path, load_config

cfg = load_config("config.yaml")
fever_data = load_jsonl_from_path(cfg['path']['fever_path'])


In [2]:
len(fever_data)

145449

In [3]:
fever_data[0]

# evidence 숫자 뭔지 확인해보기

{'id': 75397,
 'verifiable': 'VERIFIABLE',
 'label': 'SUPPORTS',
 'claim': 'Nikolaj Coster-Waldau worked with the Fox Broadcasting Company.',
 'evidence': [[[92206, 104971, 'Nikolaj_Coster-Waldau', 7],
   [92206, 104971, 'Fox_Broadcasting_Company', 0]]]}

In [4]:
fever_data[1]

{'id': 150448,
 'verifiable': 'VERIFIABLE',
 'label': 'SUPPORTS',
 'claim': 'Roman Atwood is a content creator.',
 'evidence': [[[174271, 187498, 'Roman_Atwood', 1]],
  [[174271, 187499, 'Roman_Atwood', 3]]]}

In [6]:
from collections import Counter

def check(all_data):
    ver = []
    label = []
    for i in all_data:
        ver.append(i['verifiable'])
        label.append(i['label'])
    return ver, label
ver, label = check(fever_data)




In [7]:
counter = Counter(ver)
print('')
for key in counter.keys():
    print('{:<2} : {}'.format(key, counter[key]))
print(f'total rows : {sum(counter.values())}')


VERIFIABLE : 109810
NOT VERIFIABLE : 35639
total rows : 145449


In [8]:
counter = Counter(label)
print('')
for key in counter.keys():
    print('{:<2} : {}'.format(key, counter[key]))
print(f'total rows : {sum(counter.values())}')


SUPPORTS : 80035
REFUTES : 29775
NOT ENOUGH INFO : 35639
total rows : 145449


# 5. Bias in Bios

In [7]:
!git clone https://github.com/microsoft/biosbias.git

Cloning into 'biosbias'...


In [10]:
!python biosbias\download_bios.py

usage: download_bios.py [-h] [-o OUT] [-r RETRIES] [-p N] wetpaths
download_bios.py: error: the following arguments are required: wetpaths


현재 서버 닫혀있음 -> 이메일로 열어달라고 요청한 상태

In [1]:
from datasets import load_dataset


dataset_name = "LabHC/bias_in_bios"  # 여기에 실제 데이터셋 이름을 넣어주세요.


dataset = load_dataset(dataset_name)


print(dataset)


Extracting data files:   0%|          | 0/3 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/257478 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/99069 [00:00<?, ? examples/s]

Generating dev split:   0%|          | 0/39642 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['hard_text', 'profession', 'gender'],
        num_rows: 257478
    })
    test: Dataset({
        features: ['hard_text', 'profession', 'gender'],
        num_rows: 99069
    })
    dev: Dataset({
        features: ['hard_text', 'profession', 'gender'],
        num_rows: 39642
    })
})


In [3]:
type(dataset)

datasets.dataset_dict.DatasetDict

In [6]:
dataset['train'][1]

{'hard_text': 'She is able to assess, diagnose and treat minor illness conditions and exacerbations of some long term conditions. Her qualifications include Registered General Nurse, Bachelor of Nursing, Diploma in Health Science, Emergency Care Practitioner and Independent Nurse Prescribing.',
 'profession': 13,
 'gender': 1}

In [10]:
dataset['train'][2]

{'hard_text': 'Prior to law school, Brittni graduated magna cum laude from DePaul University in 2011 with her Bachelor’s Degree in Psychology and Spanish. In 2014, she earned her law degree from Chicago-Kent College of Law. While at Chicago-Kent, Brittni was awarded two CALI Excellence for the Future Awards in both Legal Writing and for her seminar article regarding President Obama’s executive action, Deferred Action for Childhood Arrivals.',
 'profession': 2,
 'gender': 1}

In [7]:
import json
with open('biasbios_train.jsonl', 'w') as jsonl_file:
    for item in dataset['train']:
        json.dump(item, jsonl_file)
        jsonl_file.write('\n')

In [8]:
import json
with open('biasbios_test.jsonl', 'w') as jsonl_file:
    for item in dataset['test']:
        json.dump(item, jsonl_file)
        jsonl_file.write('\n')

In [9]:
import json
with open('biasbios_dev.jsonl', 'w') as jsonl_file:
    for item in dataset['dev']:
        json.dump(item, jsonl_file)
        jsonl_file.write('\n')

# 6.McRae

얘는 직접 드라이브 액세스 열어달라고 홈페이지에 요청해야됨
-> 요청한 상태

In [2]:
import pandas as pd
from tqdm import tqdm
df = pd.read_excel("df_obj_true_full_rel.xlsx")

count = 0
count_2 = 0
for i in tqdm(range(len(df))):

    if df.loc[i, 'obj_one_hop'] == 'null':
        count += 1
    elif pd.isna(df.loc[i, 'obj_one_hop']):
        count_2 += 1

print(count, 'and', count_2)

100%|██████████| 21919/21919 [00:00<00:00, 58093.73it/s]

0 and 2769


In [10]:
(df.loc[140, 'obj_one_hop'])


nan